<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт библиотек

In [68]:
import pandas as pd
import torch
import transformers 
import transformers as ppb # pytorch transformers

import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from tqdm import notebook

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

from sklearn.metrics import accuracy_score 
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

import requests
import io

from sklearn.svm import SVC
from urllib.parse import urlencode 



### Подготовка констант

In [35]:
RANDOM_STATE = 100

### Знакомство с данными

In [36]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv(r'C:\Users\maxpe\Downloads\Practicum\Projects\datasets\toxic_comments.csv')
display(df.shape);

(159292, 3)

In [37]:
# df = pd.read_csv('toxic_comments.csv')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [39]:
df

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


### Подготовка данных

Датасет не содержит пустых строк, состоит из 3 столбцов, один из которых будет преобразован в признаки (`text`), а `toxic` - целевой признак. Столбец `Unnamed` можем удалить. 

In [40]:
df = df.drop(['Unnamed: 0'], axis=1)
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


#### Загрузка предобученных модели и токенизатора.

In [41]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights);

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Проверим любой рандомный текст.

In [42]:
tokenizer.encode('You, sir, are my hero. Any chance you remember..', add_special_tokens=True) 


[101,
 2017,
 1010,
 2909,
 1010,
 2024,
 2026,
 5394,
 1012,
 2151,
 3382,
 2017,
 3342,
 1012,
 1012,
 102]

Токенизатор отрабатывает корректно. Применим его предварительно к датасету.

#### Преобразуем текст

In [43]:
pre_tokenized = df['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True)) 

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors


In [44]:
pre_tokenized

0         [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1         [101, 1040, 1005, 22091, 2860, 999, 2002, 3503...
2         [101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...
3         [101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...
4         [101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...
                                ...                        
159287    [101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...
159288    [101, 2017, 2323, 2022, 14984, 1997, 4426, 200...
159289    [101, 13183, 6290, 26114, 1010, 2045, 2015, 20...
159290    [101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...
159291    [101, 1000, 1998, 1012, 1012, 1012, 1045, 2428...
Name: text, Length: 159292, dtype: object

Как видим из сообщения к `pre_tokenized`, имеем ограничение в 512 токенов в строке. Создадим такое ограничение.

#### Фильтруем данные

Собираем датасет из токенизированных данных - столбец `text`, целевого признака и длин списков токенов.

In [45]:
df_temp = pd.DataFrame(pre_tokenized)
df_temp['toxic'] = df['toxic']
len_list = []
for i in df_temp['text']:
    len_list.append(len(i))
df_temp['len'] = len_list
df_temp

,text,toxic,len
0,"[101, 7526, 2339, 1996, 10086, 2015, 2081, 210...",0,68
1,"[101, 1040, 1005, 22091, 2860, 999, 2002, 3503...",0,35
2,"[101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...",0,54
3,"[101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...",0,144
4,"[101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...",0,21
...,...,...,...
159287,"[101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...",0,68
159288,"[101, 2017, 2323, 2022, 14984, 1997, 4426, 200...",0,27
159289,"[101, 13183, 6290, 26114, 1010, 2045, 2015, 20...",0,19
159290,"[101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...",0,28


Удаляем строки с превышающими лимит токенами. таких строк порядка 1.8%

In [46]:
df_temp = df_temp.loc[df_temp['len'] < 513].drop(['len'], axis=1)
df_temp

,text,toxic
0,"[101, 7526, 2339, 1996, 10086, 2015, 2081, 210...",0
1,"[101, 1040, 1005, 22091, 2860, 999, 2002, 3503...",0
2,"[101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...",0
3,"[101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...",0
4,"[101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...",0
...,...,...
159287,"[101, 1000, 1024, 1024, 1024, 1024, 1024, 1998...",0
159288,"[101, 2017, 2323, 2022, 14984, 1997, 4426, 200...",0
159289,"[101, 13183, 6290, 26114, 1010, 2045, 2015, 20...",0
159290,"[101, 1998, 2009, 3504, 2066, 2009, 2001, 2941...",0


Собираем данные по индексу: слева - тексты из исходного датасета, справа - целевой признак из нового, отфильтрованного датасета. Так как `join` имеет значение `inner`, то остаются только те строки, в которых совпадает индекс, то есть прошедшие фильтрацию.

In [47]:
df_left = df['text']
df_right = df_temp['toxic']
result = pd.concat([df_left, df_right], axis=1, join="inner")
result


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


#### Ограничиваем размер выборки

Для экономии ресурсов, ограничим исходную выборку. Количество строк поместим в переменную `q_sample`.

In [48]:
q_sample = 20000
df_cut = result.sample(q_sample).reset_index(drop=True)
df_cut

,text,toxic
0,", I replied in the Talk page. First I sent the...",0
1,"Now I have already emigrated from Germany, and...",0
2,"No, the game is too linear to be OW.",0
3,RESPONSE TO DEKISUGI: :The fact that you persi...,0
4,1. Active serive and ASU are commonly used ter...,0
...,...,...
19995,WTMM and WEEV\nThanks of taking care of this f...,0
19996,Peel Police Censorship and Vandalism \n\nAmazi...,0
19997,Merge with Media Theory revisit \nJust to giv...,0
19998,I hate you \nMy name is Sean Noyes and I hate ...,1


#### Преобразуем текст предобработанных данных

Токенизируем отфильтрованную и урезанную выборку.

In [49]:
tokenized = df_cut['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True)) 

In [50]:
tokenized

0        [101, 1010, 1045, 3880, 1999, 1996, 2831, 3931...
1        [101, 2085, 1045, 2031, 2525, 11367, 2013, 276...
2        [101, 2053, 1010, 1996, 2208, 2003, 2205, 7399...
3        [101, 3433, 2000, 2139, 14270, 15916, 2072, 10...
4        [101, 1015, 1012, 3161, 14262, 3512, 1998, 200...
                               ...                        
19995    [101, 1059, 21246, 2213, 1998, 16776, 2615, 42...
19996    [101, 14113, 2610, 15657, 1998, 3158, 9305, 29...
19997    [101, 13590, 2007, 2865, 3399, 7065, 17417, 21...
19998    [101, 1045, 5223, 2017, 2026, 2171, 2003, 5977...
19999    [101, 4067, 2017, 2005, 23781, 2007, 16948, 10...
Name: text, Length: 20000, dtype: object

#### Ищем размер вектора

Предупреждения больше нет. Дополнительно проверяем самое большое значение длины токенов в тексте (хоть ограничение равно 512, не факт, что такие векторы могут попасть в выборку), и создаем переменную `max_len`.

In [51]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

512

#### Преобразование векторов

Паддингом добавляем к каждому вектору нули в конец, так, чтобы длина каждого вектора была равна длине `max_len`.

Маска тоже применяется к каждому вектору - значению, отличному от нуля, присваивается 1, нули остаются нулями.

In [52]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

<a id="first">[q_sample](#1)</a>

**Расчет ячейки ниже занимает очень много времени.** 

Можно уменьшить количество строк переменной `q_sample` для проверки работоспособности кода. Но для удобства проверки, увеличения скорости работы, все заранее рассчитано на более крупной выборке, выгружено на Гитхаб/Яндекс.Диск, и загружено обратно в код.

Если делать по прямому пути (преобразование в эмбенддинги внутри ноутбука), то ячейки выполняются последовательно, кроме пункта "Загрузка датасета". Выгрузка датасета закомментирована, она не должна исполняться.

Если выполнять с заранее преобразованными эмбеддингами по второму пути, то пункт "**Преобразование в эмбеддинги**" не исполнять, и перехоить к пункту "**Загрузка датасета**". В датасете Гитхаба лежит выборка на 2000 строк, в Яндекс.Диске - на 20000.

Сейчас пункт "**Преобразование в эмбеддинги**" закомментирован во избежание неразберихи, в случае запуска всего файла целиком, а не по ячейкам. То есть, фактически, код исполняется по второму сценарию, через Яндекс.Диск. 

*Для раскомментирования/комментирования всей ячейки использовать сочетание `Ctrl + /`.*

#### Преобразование в эмбеддинги

In [53]:
# batch_size = 500 # задаем размер батча (кратен q_sample, если уменьшаем q_sample - уменьшаем и батч)
# embeddings = [] # список эмбеддингов
# for i in notebook.tqdm(range(padded.shape[0] // batch_size)): # цикл по батчам, отображать прогресс будет функция notebook()
#         batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) # преобразуем данные в формат тензоров
#         attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]) # преобразуем маску
        
#         with torch.no_grad(): # для ускорения вычисления функцией no_grad() в библиотеке torch укажем, что градиенты не нужны
#             batch_embeddings = model(batch, attention_mask=attention_mask_batch) # чтобы получить эмбеддинги для батча, передадим модели данные и маску
        
#         embeddings.append(batch_embeddings[0][:,0,:].numpy()) # преобразуем элементы методом numpy() к типу numpy.array

# df_ed = pd.DataFrame(np.concatenate(embeddings)) # соберём все эмбеддинги в матрицу признаков вызовом функции concatenate()
# df_ed['toxic'] = df_cut['toxic'] # добавляем целевой признак
# df_ed

  0%|          | 0/40 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,toxic
0,-0.210253,-0.088105,0.373194,0.045138,-0.412036,0.115870,0.592757,0.490689,0.245250,-0.675427,...,-0.206053,0.073777,-0.020319,-0.036723,0.048516,-0.063282,-0.291686,0.131764,0.505391,0
1,0.086958,0.619067,0.029847,-0.391548,-0.167374,-0.407611,0.808926,0.458772,-0.011550,-0.476740,...,-0.212916,-0.214355,-0.239323,0.307829,0.141115,0.046230,-0.242870,0.519338,0.629348,0
2,0.010980,-0.178602,0.077921,0.065314,-0.412028,-0.527476,0.038064,0.252827,0.327264,-0.307346,...,-0.057288,0.267838,-0.116387,-0.286996,-0.009243,0.093149,-0.120039,0.309154,0.658118,0
3,-0.088517,-0.471235,-0.667682,-0.059110,-0.605237,-0.350114,0.480405,0.203409,0.070762,-0.420400,...,-0.670664,-0.147321,-0.092641,0.043940,-0.066983,-0.083624,-0.187160,-0.075435,0.534236,0
4,0.146267,0.062430,-0.461665,-0.140508,-0.031926,-0.329088,0.564474,0.623633,0.113733,-0.299472,...,-0.186488,0.117171,-0.234970,-0.034679,0.198808,0.094138,-0.288756,0.299675,0.468383,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-0.538163,-0.293847,0.561924,-0.284176,-0.207451,-0.663484,0.672475,0.591221,0.425575,-0.468169,...,-0.174168,-0.147563,-0.639911,0.613554,0.122093,-0.009356,-0.035191,0.724588,0.733067,0
19996,-0.572489,0.238362,0.015432,0.161005,-0.124715,-0.135626,0.207103,0.489043,-0.259537,-0.259918,...,-0.199020,-0.188011,-0.414878,0.677697,0.208532,-0.430124,-0.331739,0.082620,0.545732,0
19997,0.067305,-0.394596,-0.363171,0.017226,-0.605997,-0.646271,0.133162,0.700449,0.066573,-0.308690,...,0.062487,-0.124399,-0.611043,0.026428,0.270664,-0.310393,-0.391263,0.631866,0.689399,0
19998,-0.006548,0.250921,-0.315235,-0.377749,-0.533632,-0.221953,0.650980,0.918593,0.315673,-0.316328,...,-0.070844,-0.102762,-0.321104,0.476053,-0.009053,-0.083950,-0.336701,0.266582,0.735228,1


### Выгрузка датасета

Выгружаем датасет после этапа преобразования в эмбеддинги, для того, чтобы не пересчитывать их в следующий сеанс или на другом устройстве.

In [54]:
# df_ed.to_csv('df_ed.csv')

### Загрузка датасета

#### Загрузка из GitHub

Рассчитанные эмбендинги я поместил на нескольких ресурсах: Гитхаб и Яндекс.Диск. В Гитхабе имеется ограничение на размер файла, поэтому код ниже использовался/используется для быстрых загрузок небольшой выборки (порядка 2000-3000 строк), для оценки метрики `F1`.

In [67]:
# # Если преобразование в эмбенддинги делается внутри тетради, эту ячейку закомментировать/не исполнять.

# # загрузка csv файла из GitHub
# #url = "https://raw.githubusercontent.com/rustyt0aster/samples/main/df_ed.csv" # обязательно в формате raw. 20 000 не вписались в ограничение 25 мегабайт. итоговый вышел на 170 МБ.
# download = requests.get(url).content

# df_ed = pd.read_csv(io.StringIO(download.decode('utf-8'))) # чтение csv таблицы и удаление образованного столбца
# df_ed = df_ed.drop(['Unnamed: 0'], axis=1)

# df_ed # вывод датафрейма

#### Загрузка из Яндекс.Диска

На данный момент используется выборка на 20000 строк, с размером файла более чем в 170 МБ. Поэтому, можем использовать Яндекс.Диск для чтения `csv-файла`.

In [75]:
# используем api 
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' 
public_key = 'https://disk.yandex.ru/d/8emThoENQmlsTA' 
 
# получаем url 
final_url = base_url + urlencode(dict(public_key=public_key)) 
response = requests.get(final_url) 
download_url = response.json()['href'] 
 
# загружаем файл в df 
download_response = requests.get(download_url)
df_ed = pd.read_csv(download_url, sep=',')
df_ed = df_ed.drop(['Unnamed: 0'], axis=1)
df_ed

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,toxic
0,-0.210253,-0.088105,0.373194,0.045138,-0.412036,0.115870,0.592757,0.490689,0.245250,-0.675427,...,-0.206053,0.073777,-0.020319,-0.036723,0.048516,-0.063282,-0.291686,0.131764,0.505391,0
1,0.086958,0.619067,0.029847,-0.391548,-0.167374,-0.407611,0.808926,0.458772,-0.011550,-0.476740,...,-0.212916,-0.214355,-0.239323,0.307829,0.141115,0.046230,-0.242870,0.519338,0.629348,0
2,0.010980,-0.178602,0.077921,0.065314,-0.412028,-0.527477,0.038064,0.252827,0.327264,-0.307346,...,-0.057288,0.267838,-0.116387,-0.286996,-0.009243,0.093149,-0.120039,0.309154,0.658118,0
3,-0.088517,-0.471235,-0.667682,-0.059110,-0.605237,-0.350114,0.480405,0.203409,0.070762,-0.420400,...,-0.670664,-0.147321,-0.092641,0.043940,-0.066983,-0.083624,-0.187160,-0.075435,0.534236,0
4,0.146267,0.062430,-0.461665,-0.140508,-0.031926,-0.329088,0.564474,0.623633,0.113733,-0.299472,...,-0.186488,0.117171,-0.234970,-0.034679,0.198808,0.094138,-0.288756,0.299675,0.468383,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-0.538163,-0.293847,0.561924,-0.284176,-0.207451,-0.663484,0.672475,0.591221,0.425575,-0.468169,...,-0.174168,-0.147563,-0.639911,0.613554,0.122093,-0.009356,-0.035191,0.724588,0.733067,0
19996,-0.572489,0.238362,0.015432,0.161005,-0.124715,-0.135626,0.207103,0.489043,-0.259537,-0.259918,...,-0.199020,-0.188011,-0.414878,0.677697,0.208532,-0.430124,-0.331739,0.082619,0.545732,0
19997,0.067305,-0.394596,-0.363171,0.017226,-0.605997,-0.646271,0.133162,0.700449,0.066573,-0.308690,...,0.062487,-0.124399,-0.611043,0.026428,0.270664,-0.310393,-0.391263,0.631866,0.689399,0
19998,-0.006548,0.250921,-0.315235,-0.377749,-0.533632,-0.221953,0.650980,0.918593,0.315673,-0.316328,...,-0.070844,-0.102762,-0.321104,0.476053,-0.009053,-0.083950,-0.336701,0.266582,0.735228,1


## Обучение

### Разбиение данных

In [55]:
features_train, features_test, target_train, target_test = train_test_split( #создаем 4 датасета, два признаков (тест+валидация) и два целевых, 
    df_ed.drop(columns='toxic'), #для датасетов признаков удаляем целевой
    df_ed['toxic'], #для целевого оставляем только целевой
    test_size=0.2, #с соотношением 75/25
    random_state=RANDOM_STATE, #с заданной опорой для рандома 
    stratify= df_ed['toxic']) #с заданной стратификацией по целевому признаку

### Модель логистической регрессии

In [56]:
%%time

log_reg = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight= 'balanced') #максимальное количество итераций 1000, 
log_reg_scores = cross_val_score(log_reg, features_train, target_train.values, scoring='f1', cv=10)
log_reg_f1 = log_reg_scores.mean()

print("Среднее значение метрики F1 для модели логистической регрессии "\
    "с использованием кросс-валидации:", log_reg_f1)

c:\Users\maxpe\anaconda3\envs\ds_practicum_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Среднее значение метрики F1 для модели логистической регрессии с использованием кросс-валидации: 0.6322827525389966
CPU times: total: 39.7 s
Wall time: 54.2 s


In [57]:
# log_reg.fit(features_train, target_train) #обучение модели на тренировочных данных
# predictions_log_reg = log_reg.predict(features_test) #предсказание целевого признака на основе тренировочных признаков
# result_log_reg = f1_score(target_test, predictions_log_reg) #качество методом score на основе предсказанных данных и 

# result_log_reg

### Модель опорных векторов

In [58]:
%%time

svclassifier = SVC(kernel='linear') #максимальное количество итераций 1000, 
svclassifier_scores = cross_val_score(svclassifier, features_train, target_train.values, scoring='f1', cv=10)
svclassifier_f1 = svclassifier_scores.mean()

print("Среднее значение метрики F1 для модели SVC "\
    "с использованием кросс-валидации:", svclassifier_f1)

Среднее значение метрики F1 для модели логистической регрессии с использованием кросс-валидации: 0.6834586100172242
CPU times: total: 2min 18s
Wall time: 2min 21s


### Модель случайного леса

In [59]:
# score = make_scorer(f1_score, greater_is_better=True)

In [60]:
%%time

forest = RandomForestClassifier(random_state=RANDOM_STATE) #модель случайного леса
parameters = {'n_estimators': range (1, 500), 'max_depth': range (1, 100)} #перебор гиперпараметров
#применение метода гридсёрч со встроенной кросс-валидацией к модели леса с перебором указанных параметров

randomized_forest = RandomizedSearchCV(forest, n_iter=20, param_distributions= parameters, scoring='f1', n_jobs= -1, cv=5)
#обучение модели
randomized_forest.fit(features_train, target_train.values)

#лучшее значение после перебора параметров 
best_forest = abs(randomized_forest.best_score_)

print("Лучшие параметры для модели случайного леса с "\
    "использованием кросс-валидации:", randomized_forest.best_params_)
print("Наибольшее значение метрики F1 для модели случайного леса "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_forest)

Лучшие параметры для модели случайного леса с использованием кросс-валидации: {'n_estimators': 101, 'max_depth': 58}
Наибольшее значение метрики F1 для модели случайного леса при лучших гиперпараметрах с использованием кросс-валидации: 0.41778114470409805
CPU times: total: 41.2 s
Wall time: 12min 49s


### LightGBM

In [61]:
%%time

lgbm = lgb.LGBMClassifier() #
parameters = {'n_estimators': range (1, 1000), 'max_depth': range (1, 100), 'learning_rate': (0.05, 0.5, 0.05)}
#применение метода гридсёрч со встроенной кросс-валидацией

rand_lgbm = RandomizedSearchCV(lgbm, n_iter=20, param_distributions= parameters, scoring='f1', n_jobs= -1, cv=5)
#обучение модели
rand_lgbm.fit(features_train, target_train.values)

#лучшее значение после перебора параметров 
best_lgbm = rand_lgbm.best_score_

print("Лучшие параметры для модели случайного леса с "\
    "использованием кросс-валидации:", rand_lgbm.best_params_)
print("Наибольшее значение метрики F1 для модели LGBMClassifier "\
    "при лучших гиперпараметрах с использованием кросс-валидации:", best_lgbm)

Лучшие параметры для модели случайного леса с использованием кросс-валидации: {'n_estimators': 844, 'max_depth': 45, 'learning_rate': 0.5}
Наибольшее значение метрики F1 для модели LGBMClassifier при лучших гиперпараметрах с использованием кросс-валидации: 0.6724757980296925
CPU times: total: 1min 31s
Wall time: 17min 8s


## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны